In [35]:
import networkx as nx
import pandas as pd
import numpy as np
import scipy as sp

In [7]:
df_edges = pd.read_csv('../data/edgelist.csv')
df_edges

,source,source_score,target,target_score,weight,subreddit
0,Kostik114,1027,KayAyeeWhy,1757,302.000000,dogecoin
1,No-Relationship-5985,687,KayAyeeWhy,1757,229.000000,dogecoin
2,No-Relationship-5985,687,Kostik114,1027,0.758278,dogecoin
3,Sarderiol,453,KayAyeeWhy,1757,170.000000,dogecoin
4,Sarderiol,453,Kostik114,1027,0.562914,dogecoin
...,...,...,...,...,...,...
41093,aoskiev,3,TheWorldofGood,331,3.000000,btc
41094,geogrant1000,12,Michellerose6834,593,3.000000,CryptoCurrencies
41095,BasicallyTony,3,Michellerose6834,593,3.000000,CryptoCurrencies
41096,BasicallyTony,3,geogrant1000,12,1.000000,CryptoCurrencies


In [16]:
G = nx.from_pandas_edgelist(df_edges, create_using=nx.DiGraph())

In [88]:
nx_ranks = nx.pagerank(G)

In [109]:
def page_rank(G: nx.digraph, iterations=100, alpha=0.85, error=1.0e-6):
    if len(G)==0: return {}
    nodes = list(G)
    A = nx.to_numpy_array(G, nodelist=nodes, weight="weight", dtype=float)
    n, m = A.shape
    if n==0: return {}
    S = A.sum(axis=1)
    S[S != 0] = 1.0 / S[S != 0]
    A = sp.sparse.csr_array(sp.sparse.spdiags(S.T, 0, *A.shape)).dot(A)
    x = np.ones(n)/n
    p = np.ones(n)/n
    for _ in range(iterations):
        xlast = x
        x = alpha * (x.dot(A) + sum(x[np.where(S == 0)[0]]) * p) + (1 - alpha) * p
        err = np.absolute(x - xlast).sum()
        if err < n * error:
            return dict(zip(nodes, map(float, x)))
    return dict(zip(nodes,map(float,x)))

In [110]:
own_ranks = page_rank(G)

In [111]:
overall_error = np.array([np.absolute(own_ranks[k]-nx_ranks[k]) for k in own_ranks.keys()])

In [112]:
overall_error.sum()

3.189112927730331e-16